## sqlparseを触ってみた
* sqlparseによるSQLのフォーマット : https://qiita.com/atsaki/items/eb36a6a7935d525a43e7

In [1]:
import sqlparse

In [2]:
sql = '''
select 
  user
  ,country 
from hoge.table as a 
left join fuga.table as b
on a.id = b.id 
where type = "HOGE"
'''

In [9]:
print(sqlparse.format(sql, reindent=True, keyword_case='lower'))

'\nselect user ,\n       country\nfrom hoge.table as a\nleft join fuga.table as b on a.id = b.id\nwhere type = "HOGE"'

In [8]:
print(sqlparse.format(sql, reindent=True, keyword_case='upper'))


SELECT USER ,
       country
FROM hoge.table AS a
LEFT JOIN fuga.table AS b ON a.id = b.id
WHERE TYPE = "HOGE"


In [11]:
parsed = sqlparse.parse(sql)
stmt = parsed[0]
for t in stmt.tokens:
    print(type(t), t)

<class 'sqlparse.sql.Token'> 

<class 'sqlparse.sql.Token'> select
<class 'sqlparse.sql.Token'>  
<class 'sqlparse.sql.Token'> 

<class 'sqlparse.sql.Token'>  
<class 'sqlparse.sql.Token'>  
<class 'sqlparse.sql.IdentifierList'> user
  ,country
<class 'sqlparse.sql.Token'>  
<class 'sqlparse.sql.Token'> 

<class 'sqlparse.sql.Token'> from
<class 'sqlparse.sql.Token'>  
<class 'sqlparse.sql.Identifier'> hoge.table as a
<class 'sqlparse.sql.Token'>  
<class 'sqlparse.sql.Token'> 

<class 'sqlparse.sql.Token'> left join
<class 'sqlparse.sql.Token'>  
<class 'sqlparse.sql.Identifier'> fuga.table as b
<class 'sqlparse.sql.Token'> 

<class 'sqlparse.sql.Token'> on
<class 'sqlparse.sql.Token'>  
<class 'sqlparse.sql.Comparison'> a.id = b.id
<class 'sqlparse.sql.Token'>  
<class 'sqlparse.sql.Token'> 

<class 'sqlparse.sql.Where'> where type = "HOGE"

